## Agent With Multi-Tool

In [16]:
#!pip install langchain langchain_cohere langchain_experimental faiss-cpu tiktoken

In [20]:
import os, dotenv

dotenv.load_dotenv()

COHERE_API_KEY = os.getenv("COHERE_API_KEY")
#os.environ["TAVILY_API_KEY"] = os.getenv('TAVILY_API_KEY')

### WebSearch Using Tavily

In [21]:
from langchain_community.tools.tavily_search import TavilySearchResults

internet_search = TavilySearchResults()
internet_search.name = "internet_search"
internet_search.description = "Returns a list of relevant document snippets for a textual query retrieved from the internet."


from langchain_core.pydantic_v1 import BaseModel, Field
class TavilySearchInput(BaseModel):
    query: str = Field(description="Query to search the internet with")
internet_search.args_schema = TavilySearchInput

### Vector Store Tool

In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_cohere import CohereEmbeddings

# Set embeddings
embd = CohereEmbeddings()

# Docs to index
urls = [
    "https://paulgraham.com/best.html",
]

# Load
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorstore
vectorstore = FAISS.from_documents(
    documents=doc_splits,
    embedding=embd,
)

vectorstore_retriever = vectorstore.as_retriever()

In [23]:
from langchain.tools.retriever import create_retriever_tool

vectorstore_search = create_retriever_tool(
    retriever=vectorstore_retriever,
    name="vectorstore_search",
    description="Retrieve relevant info from a vectorstore that contains information from Paul Graham about how to write good essays."
)

### Python Interpreter Tool

In [ ]:
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()
python_tool = Tool(
    name="python_repl",
    description="Executes python code and returns the result. The code runs in a static sandbox without interactive mode, so print output or save output to a file.",
    func=python_repl.run,
)
python_tool.name = "python_interpreter"

# from langchain_core.pydantic_v1 import BaseModel, Field
class ToolInput(BaseModel):
    code: str = Field(description="Python code to execute.")
python_tool.args_schema = ToolInput


### Transforming Python Function Into Tool

In [24]:
from langchain_core.tools import tool
import random

@tool
def random_operation_tool(a: int, b: int):
  """Calculates a random operation between the inputs."""
  coin_toss = random.uniform(0, 1)
  if coin_toss > 0.5:
    return {'output': a*b}
  else:
    return {'output': a+b}

random_operation_tool.name = "random_operation" # use python case
random_operation_tool.description = "Calculates a random operation between the inputs."

from langchain_core.pydantic_v1 import BaseModel, Field
class random_operation_inputs(BaseModel):
    a: int = Field(description="First input")
    b: int = Field(description="Second input")
random_operation_tool.args_schema = random_operation_inputs

### ReACT Agent

In [25]:
from langchain.agents import AgentExecutor
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_cohere.chat_models import ChatCohere

# LLM
llm = ChatCohere(model="command-r-plus", temperature=0.3)

# Preamble
preamble = """
You are an expert who answers the user's question with the most relevant datasource. You are equipped with an internet search tool and a special vectorstore of information about how to write good essays.
You also have a 'random_operation_tool' tool, you must use it to compute the random operation between two numbers.
"""


# Prompt template
prompt = ChatPromptTemplate.from_template("{input}")

# Create the ReAct agent
agent = create_cohere_react_agent(
    llm=llm,
    tools=[internet_search, vectorstore_search, python_tool, random_operation_tool],
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=agent, tools=[internet_search, vectorstore_search, python_tool, random_operation_tool], verbose=True)

### Standalone Question to ReAct Agent

In [26]:
response = agent_executor.invoke({
    "input": "I want to write an essay about the Roman Empire. Any tips for writing an essay? Any fun facts?",
    "preamble": preamble,
})

response['output']

# note that the model smartly looks in the vector db, and then online



> Entering new AgentExecutor chain...

I will search for tips on how to write an essay and fun facts about the Roman Empire.
{'tool_name': 'vectorstore_search', 'parameters': {'query': 'tips for writing an essay'}}
be? I should have asked how do you write essays well? Though
these seem only phrasing apart, their answers diverge. The answer
to the first question, as we've seen, isn't really about essay
writing. The second question forces it to be.Writing essays, at its best, is a way of discovering ideas. How do
you do that well? How do you discover by writing?An essay should ordinarily start with what I'm going to call a
question, though I mean this in a very general sense: it doesn't
have to be a question grammatically, just something that acts like
one in the sense that it spurs some response.How do you get this initial question? It probably won't work to
choose some important-sounding topic at random and go at it.
Professional traders won't even trade unless they have what they
ca

"Here are some tips for writing an essay:\n- Start with a question that spurs some response.\n- Don't choose a topic at random, make sure you have a way in, some new insight or approach.\n- You don't need a complete thesis, just a gap to explore.\n- Write down your initial response, which is usually mistaken or incomplete. Writing converts your ideas from vague to bad, but that's a step forward because once you can see the brokenness, you can fix it.\n- Keep rereading what you've written and asking yourself if it's correct.\n- You can get ideas by talking to people, reading, doing and building things, and going places and seeing things.\n\nHere are some fun facts about the Roman Empire:\n- At its peak, the empire stretched from North Africa to Britain, reigning over 60 million people.\n- The story of Rome's warrior founder and the twins and their wolf step-mother was included in the first history of the city, written by the Greek writer Diocles of Peparethus.\n- The Crossing of the Rhi

In [27]:
response = agent_executor.invoke({
    "input": "Calculate the result of the random operation of 10 and 20. Then find a few fun facts about that number, as well as its prime factors.",
    "preamble": preamble,
})

response['output']



> Entering new AgentExecutor chain...

First, I will calculate the result of the random operation between 10 and 20. Then, I will search for fun facts about that number and its prime factors.
{'tool_name': 'random_operation_tool', 'parameters': {'a': 10, 'b': 20}}
random_operation_tool is not a valid tool, try one of [internet_search, vectorstore_search, python_interpreter, random_operation].
I have received an error message. I will now try to use the random_operation tool to calculate the result of the random operation between 10 and 20.
{'tool_name': 'random_operation', 'parameters': {'a': 10, 'b': 20}}
{'output': 200}
The result of the random operation between 10 and 20 is 200. Now, I will search for fun facts about the number 200 and its prime factors.
{'tool_name': 'internet_search', 'parameters': {'query': 'fun facts about the number 200'}}
[{'url': 'https://www.archimedes-lab.org/numbers/Num70_200.html', 'content': "With 189 pages filled with an incredible variety of fun facts

"The result of the random operation between 10 and 20 is **200**. Here are some fun facts about the number 200:\n- It is an even composite number composed of two distinct primes.\n- It is the smallest base 10 unprimeable number, meaning it cannot be turned into a prime number by changing just one of its digits to any other digit.\n- According to Bullinger's study of biblical literature, the number 200 signifies 'insufficiency'.\n\nThe prime factors of 200 are 2x2x2 and 5x5."

### A question that requires the large language model to directly answer.

In [28]:
response = agent_executor.invoke({
    "input": "Hey how are you?",
    "preamble": preamble,
})

response['output']



> Entering new AgentExecutor chain...

I will respond to the user's greeting.
{'tool_name': 'directly_answer', 'parameters': {}}
directly_answer is not a valid tool, try one of [internet_search, vectorstore_search, python_interpreter, random_operation].Relevant Documents: None
Cited Documents: None
Answer: I'm an AI assistant, so I don't have feelings as such, but I'm here to help you with your queries. How can I help?
Grounded answer: I'm an AI assistant, so I don't have feelings as such, but I'm here to help you with your queries. How can I help?

> Finished chain.


"I'm an AI assistant, so I don't have feelings as such, but I'm here to help you with your queries. How can I help?"

### Ask a more complex question to the ReAct agent

In [29]:
response = agent_executor.invoke({
    "input": "In what year was the company that was founded as Sound of Music went public? What was its stock price in 2000 and 2010.",
    "preamble": preamble,
})

response['output']



> Entering new AgentExecutor chain...

I will first search for the company that was founded as Sound of Music. Then, I will search for the year it went public. Finally, I will search for its stock price in 2000 and 2010.
{'tool_name': 'internet_search', 'parameters': {'query': 'company founded as Sound of Music'}}
[{'url': 'https://en.wikipedia.org/wiki/Best_Buy', 'content': 'Under the Geek Squad brand, Best Buy offers computer repair, warranty service, and accidental service plans.[2] Best Buy provides an online community forum for members, where consumers can discuss product experiences, ask questions, and get answers from other members or retail product experts.[82]\nThe building exteriors of Best Buy-branded stores are typically light brown, with the entrance designed to look like a blue box emerging from the structure.[83] Corporate employees operated under a results only work environment from 2005 until March 2013, when the management style was abandoned by Best Buy CEO Hubert 

'Sound of Music was renamed Best Buy Company, Inc. in 1983. The company went public in 1985 and debuted on the New York Stock Exchange in 1987. I was unable to find the stock price for Best Buy in 2000 and 2010. However, I found that the latest closing stock price for Best Buy as of April 1, 2024, is 81.64.'

Conversation with Multi-Agent ReACT

In [30]:
# Step 1: Construct the chat history as a list of LangChain Messages, ending with the last user message
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(content="I'm considering switching to Oracle for my CRM."),
    AIMessage(content="That sounds like a good idea! How can I help you?"),
    HumanMessage(content="Recap me all the info you can find about their offering."),
]

prompt = ChatPromptTemplate.from_messages(chat_history)

In [31]:
# Step 2: When you make the agent, specify the chat_history as the prompt, e.g.
# Create the ReAct agent
agent = create_cohere_react_agent(
    llm=llm,
    tools=[internet_search, vectorstore_search, python_tool],
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=agent, tools=[internet_search, vectorstore_search, python_tool], verbose=True)
  

In [ ]:
# Step 3: When you invoke the agent_executor there's no need to pass anything else into invoke
response = agent_executor.invoke({
    "preamble": preamble,
})

response['output']



> Entering new AgentExecutor chain...

I will search for 'Oracle CRM offering' and relay the information I find to the user.
{'tool_name': 'internet_search', 'parameters': {'query': 'Oracle CRM offering'}}
[{'url': 'https://www.trustradius.com/products/oracle-crm-on-demand/reviews?qs=pros-and-cons', 'content': "What is Oracle CRM On Demand?The basis of this offering is the Market2Lead product that Oracle acquired in 2010. It has now been fully integrated with Oracle's On Demand CRM product and is a full-featured marketing automation product with features from lead management and nurturing, to measuring marketing ROI."}, {'url': 'https://www.softwareadvice.com/resources/breaking-down-oracle-crm/', 'content': "Oracle's Marketing Cloud provides a comprehensive set of tools that cover a range of digital marketing needs. This includes tools for cross-channel marketing, i.e., marketing automation for both B2B and B2C marketers, as well as data management, content marketing and social media

"Oracle CRM is a customer relationship management system created by Oracle Corporation. It includes a number of different cloud applications that can be deployed together or used individually to analyse customer data and help companies connect and manage sales, marketing and customer support. \n\nOracle CRM is based on the Market2Lead product that Oracle acquired in 2010. It has now been fully integrated with Oracle's On Demand CRM product and is a full-featured marketing automation product with features from lead management and nurturing, to measuring marketing ROI. \n\nOracle's Marketing Cloud provides a comprehensive set of tools that cover a range of digital marketing needs. This includes tools for cross-channel marketing, i.e., marketing automation for both B2B and B2C marketers, as well as data management, content marketing and social media marketing. \n\nThe next-generation Oracle CRM experience puts the seller's ever-changing role at its core. It provides a prescriptive, intell